In [2]:
import requests
import datetime
from dateutil.relativedelta import relativedelta  # to add days or years
from datetime import date
import streamlit as st
import requests
import pandas as pd
import plotly.graph_objects as go
from bokeh.models.widgets import Div

In [27]:
query = 'Technology'
params = dict(q=query)

today = datetime.datetime.now()
limit_date = today + relativedelta(days=-7)
today_time = today.timestamp()
limit_time = limit_date.timestamp()



tweet_params = dict(q=query,
                    filters=f"timestamp < {limit_time}",
                    limit=100000)
tweet_params_without_query = dict(q="",
                                  filters=f"timestamp < {limit_time}")

headers = {'X-Meili-API-Key': 'OTkwNzQ0ZGRkZTc0NDcwM2RlMzFlOGIx'}





def get_consultations():
    lexai_url = "http://127.0.0.1:7700/indexes/consultations/search"
    result = requests.get(lexai_url, params=params, headers=headers).json()
    consultations = []
    for i in result["hits"]:
        title = i['title']
        topics = i['topics']
        type_of_act = i['type_of_act']
        try:
            status = i["status"]
        except:
            status= ""
        try:
            end_date = pd.to_datetime(i['end_date'],errors='coerce').date()
        except:
            end_date = pd.to_datetime(i['end_date'], errors='coerce')
        link = i['link']
        consultations.append({
            "title": title,
            "status": status,
            "topics": topics,
            "type_of_act": type_of_act,
            "end_date": end_date,
            "link": link
        })

    return consultations

In [28]:
result = get_consultations()
result


[{'title': 'EU cooperation on Health Technology Assessment',
  'status': '',
  'topics': 'Food safety',
  'type_of_act': 'PROP_REG',
  'end_date': datetime.date(2018, 4, 2),
  'link': 'https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/1565-EU-cooperation-on-Health-Technology-Assessment_en'},
 {'title': 'Amendment of the Regulation on the European Institute of Innovation and Technology (EIT)',
  'status': '',
  'topics': 'Culture and media',
  'type_of_act': 'PROP_REG',
  'end_date': datetime.date(2019, 9, 5),
  'link': 'https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/1530-Amendment-of-the-Regulation-on-the-European-Institute-of-Innovation-and-Technology-(EIT)_en'},
 {'title': 'Strategic Innovation Agenda for the period 2021-2027 of the European Institute of Innovation and Technology (EIT)',
  'status': '',
  'topics': 'Culture and media',
  'type_of_act': 'PROP_DEC',
  'end_date': datetime.date(2019, 9, 5),
  'link': 'https://ec.europa.

In [17]:
for x in result:
    print(x)
    print(x["end_date"])

{'title': 'Farm Accountancy Data Network (FADN) - collecting data on farm incomes and its economic activities', 'status': '', 'topics': 'Agriculture and rural development', 'type_of_act': 'REG_IMPL', 'end_date': NaT, 'link': 'https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/12345-Farm-Accountancy-Data-Network-(FADN)---collecting-data-on-farm-incomes-and-its-economic-activities_en'}
NaT
{'title': 'Farm to Fork Strategy  ', 'status': '', 'topics': 'Food safety', 'type_of_act': 'COMMUNIC', 'end_date': NaT, 'link': 'https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/12183-Farm-to-Fork-Strategy--_en'}
NaT
{'title': 'EU farm policy - Evaluation of its impact on sustainable management of the soil ', 'status': '', 'topics': 'Agriculture and rural development', 'type_of_act': 'SWD', 'end_date': datetime.date(2020, 10, 22), 'link': 'https://ec.europa.eu/info/law/better-regulation/have-your-say/initiatives/11834-EU-farm-policy---Evaluation-of-its-im

In [40]:
#Data from News
lexai_url_news = "http://127.0.0.1:7700/indexes/twitter_press/search"
news = requests.get(lexai_url_news, params=tweet_params,
                    headers=headers).json()

#Data from Politicians
lexai_url_politicians = "http://127.0.0.1:7700/indexes/twitter_politicians/search"
politicians = requests.get(lexai_url_politicians,
                           params=tweet_params,
                           headers=headers).json()

#Data from General
lexai_url_general = f"http://127.0.0.1:7700/indexes/twitter_query/search/"
query_data_general = requests.get(lexai_url_general,
                                  params=tweet_params,
                                  headers=headers).json()

In [41]:
query_data_general

{'hits': [{'timestamp': 1622783339.0,
   'id': 1400711160509341698,
   'user': 'hanisshit',
   'text': 'tongue technology',
   'text_en': 'tongue technology',
   'date': '2021/06/04 07:08:59',
   'lang': 'cs',
   'iso_lang': 'cs',
   'user_verified': False,
   'followers_count': 8,
   'user_loc': '',
   'user_desc': "hani's shitpost acct",
   'user_desc_en': 'hani&#39;s shitpost acct',
   'user_image': 'http://pbs.twimg.com/profile_images/765017831347924992/5h7FtWhj_normal.jpg',
   'hashtags': '',
   'mentions': '',
   'retweet_count': 0,
   'favorite_count': 0,
   'link': 'https://twitter.com/hanisshit/status/1400711160509341698'},
  {'timestamp': 1622783316.0,
   'id': 1400711066246545411,
   'user': 'moothought',
   'text': 'when is technology going to reach baby translation; I have questions.',
   'text_en': 'when is technology going to reach baby translation; I have questions.',
   'date': '2021/06/04 07:08:36',
   'lang': 'en',
   'iso_lang': 'en',
   'user_verified': False,
   '

In [42]:

# Sentiment pie-charts
#All data
query_df = pd.DataFrame(query_data_general['hits'])
politicians_df = pd.DataFrame(politicians['hits'])
news_df = pd.DataFrame(news['hits'])
all_tweets_df = pd.concat([query_df, politicians_df, news_df])

colors_pie = ['#11E7FB', '#6082FD', '#731F7D']

labels = ["Positive", "Neutral", "Negative"]

values_query = [
    all_tweets_df['sentiment'].value_counts()['positive'],
    all_tweets_df['sentiment'].value_counts()['neutral'],
    all_tweets_df['sentiment'].value_counts()['negative']
]
values_all = [41, 35, 24]

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1,
                    cols=2,
                    specs=[[{
                        'type': 'domain'
                    }, {
                        'type': 'domain'
                    }]])
fig.add_trace(
    go.Pie(title='Twitter',
           labels=labels,
           values=values_all,
           direction='clockwise',
           sort=False), 1, 1)
fig.add_trace(
    go.Pie(title=query,
        labels=labels,
           values=values_query,
           direction='clockwise',
           sort=False), 1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4,
                  hoverinfo="label+percent",
                  textfont_size=15,
                  marker=dict(colors=colors_pie))

fig.update_layout(
    # Add annotations in the center of the donut pies.
    annotations=[
        dict(text='',
             x=0.18,
             y=0.5,
             font_size=18,
             font_color='#731F7D',
             showarrow=False),
        dict(text="",
             x=0.82,
             y=0.5,
             font_size=18,
             font_color='#731F7D',
             showarrow=False)], autosize=False, width=500, height=250, margin=dict(l=0,r=0,b=0,t=0,pad=1))


KeyError: 'sentiment'

In [43]:
all_tweets_df

,timestamp,id,user,text,text_en,date,lang,iso_lang,user_verified,followers_count,user_loc,user_desc,user_desc_en,user_image,hashtags,mentions,retweet_count,favorite_count,link
0,1.622783e+09,1400711160509341698,hanisshit,tongue technology,tongue technology,2021/06/04 07:08:59,cs,cs,False,8,,hani's shitpost acct,hani&#39;s shitpost acct,http://pbs.twimg.com/profile_images/7650178313...,,,0,0,https://twitter.com/hanisshit/status/140071116...
1,1.622783e+09,1400711066246545411,moothought,when is technology going to reach baby transla...,when is technology going to reach baby transla...,2021/06/04 07:08:36,en,en,False,438,Twitter-verse,( Insert Adjectives ),( Insert Adjectives ),http://pbs.twimg.com/profile_images/1375600698...,,,0,0,https://twitter.com/moothought/status/14007110...
2,1.622783e+09,1400711150413754370,scrumnl,RT @rob_england: @technology @emilychangtv Say...,RT @rob_england: @technology @emilychangtv Say...,2021/06/04 07:08:56,en,en,False,469,Room 3B,Rincewindian Post-Agilist,Rincewindian Post-Agilist,http://pbs.twimg.com/profile_images/1227853908...,,"rob_england, technology, emilychangtv",2,0,https://twitter.com/scrumnl/status/14007111504...
3,1.622783e+09,1400711119887646724,CantStopStompin,RT @veritasnewsfeed: mRNA technology pioneer s...,RT @veritasnewsfeed: mRNA technology pioneer s...,2021/06/04 07:08:49,en,en,False,18,,Never ever give up. 28. Gay. Trumplican.,Never ever give up. 28. Gay. Trumplican.,http://pbs.twimg.com/profile_images/1389693339...,,veritasnewsfeed,57,0,https://twitter.com/CantStopStompin/status/140...
4,1.622783e+09,1400711120067911683,hannahfrankbot,DOCTOR: Little bit of technology from my home.,DOCTOR: Little bit of technology from my home.,2021/06/04 07:08:49,en,en,False,0,,,None,http://abs.twimg.com/sticky/default_profile_im...,,,0,0,https://twitter.com/hannahfrankbot/status/1400...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,1.622690e+09,1400318866757128192,euobs,The partnership will target technologies with ...,None,2021/06/03 05:10:09,en,None,True,197551,Brussels,Influential. Investigative. Independent. EUobs...,None,http://pbs.twimg.com/profile_images/1345057594...,,,0,1,https://twitter.com/euobs/status/1400318866757...
7,1.622641e+09,1400112472984457223,euobs,The EU is falling behind the US and China in #...,None,2021/06/02 15:30:01,en,None,True,197551,Brussels,Influential. Investigative. Independent. EUobs...,None,http://pbs.twimg.com/profile_images/1345057594...,"ArtificialIntelligence, blockchain",EIB,0,1,https://twitter.com/euobs/status/1400112472984...
8,1.622721e+09,1400448440748945412,EURACTIV,.@EU_Commission announcement: Bill Gates-found...,None,2021/06/03 13:45:02,en,None,True,137848,Brussels,EU news & policy debates from the European Med...,None,http://pbs.twimg.com/profile_images/8235903410...,Climate,"EU_Commission, Breakthrough",0,6,https://twitter.com/EURACTIV/status/1400448440...
9,1.622617e+09,1400014358260953089,EURACTIV,🇫🇮 What will drive Helsinki's transition to #C...,🇫🇮 What will drive Helsinki's transition to #C...,2021/06/02 09:00:08,en,None,True,137846,Brussels,EU news & policy debates from the European Med...,EU news & policy debates from the European Med...,http://pbs.twimg.com/profile_images/8235903410...,CarbonNeutrality,FredSimonEU,4,3,https://twitter.com/EURACTIV/status/1400014358...


In [30]:
query_df

""


In [44]:
import json
import os
import sys
from collections import Counter
from os.path import dirname, exists, join

import gensim.downloader as api
import meilisearch
import nltk
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer

load_dotenv(dotenv_path=join(dirname(dirname(__file__)),'.env'))

class Analyse:
    
    def __init__(self, url='http://127.0.0.1:7700', key=None):
        
        if key is None:
            key = os.getenv('MEILISEARCH_KEY')
        
        self.client = meilisearch.Client(url, key)
        self.indices = [idx.get('name', None) for idx in self.client.get_indexes()]
        
    @staticmethod
    def get_words():
        text_keys = {
            'eurlex': ['title'],
            'consultations': ['title', 'topics'],
            'twitter_query': ['text_en', 'user_desc_en'],
            'twitter_politicians': ['text_en', 'user_desc_en'],
            'twitter_press': ['text_en', 'user_desc_en']
        }
        
        print('Extracting all lemmatized words from json files... ', end='')
        folder = join(os.path.abspath(''), 'data.json')
        if not exists(folder):  # if folder doesn't exist
            print('Folder not found.')
            return

        files = os.listdir(folder)
        indices = [file.replace('.json', '') for file in files]
        if len(files) == 0:  # if no files
            print('No files found.')
            return

        words = []
        stop_words = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()
        tokenizer = nltk.RegexpTokenizer(r"\w+")

        for index, filename in zip(indices, files):
            with open(join(folder, filename), 'r') as file:
                json_ = json.load(file)

            for i in range(len(json_)):
                for key in text_keys[index]:
                    if json_[i][key] is not None:
                        text = json_[i][key].lower().replace(r'\d','')
                        tokens = [lemmatizer.lemmatize(word) for word in tokenizer.tokenize(text) if word not in stop_words]
                        words.extend(tokens)
        
        print('Done.\n')
        return words
    
    def get_synonyms(self, frac=0.03, n=0.5):
        words = self.get_words()
        
        print('Loading model... ', end='')
        word2vec_transfer = api.load('word2vec-google-news-300')
        print('Done.')
        
        print('Getting available words in model... ', end='')
        available_words = list(word2vec_transfer.key_to_index.keys())
        print('Done.')
        
        print('Extracting nouns and adjectives...', end='')
        words_to_keep = set([word for (word, tag) in nltk.pos_tag(words) if any(i in tag for i in ['NN', 'JJ'])])
        print('Done.')
        
        print('Filtering words... ', end='')
        word_counts = {k: v for k, v in Counter(words).items() if k in words_to_keep}
        if isinstance(frac, int):
            min_count = frac
        else:
            min_count = sorted(list(word_counts.values()))[::-1][int(len(word_counts.values()) * 0.03)]
        words = [word for word in words_to_keep if (word_counts[word] > min_count and word in available_words)]
        print('Done.')
        
        print('Finding synonyms... ', end='')
        if n >= 1:
            synonyms = {word: list(np.array(word2vec_transfer.most_similar(word, topn=n))[:,0]) for word in words}
        else:
            synonyms = {}
            for word in words:
                arr = np.array(word2vec_transfer.most_similar(word, topn=20))
                synonyms[word] = list(arr[arr[:,1].astype(float) > n][:,0])
        print('Done.')
        
        folder = join(os.path.abspath(''), 'data.json')
        with open(join(folder, 'synonyms.json'), 'w') as file:
            json.dump(synonyms, file)
        
        print('Saved synonyms to data.json/synonyms.json\n')
        return synonyms
    
    def get_all_sentiments(self):
        folder = dirname(dirname(__file__))
        if not exists(folder):  # if folder doesn't exist
            print('Folder not found.')
            return
        
        files = os.listdir(folder)
        indices = [file.replace('.json', '') for file in files]
        
        if len(files) == 0:  # if no files
            print('No files found.')
            return
        
        for index, filename in zip(indices, files):
            filepath = join(folder, 'data.json', filename)
            with open(filepath, 'r') as file:
                documents = json.load(file)
            
            filepath = join(folder, 'update.json', f'{index}_sentiment.json')
            with open(filepath, 'w') as file:
                json.dump(self.get_sentiments(documents), file)

    
    def get_sentiments(self, data):
        #initialize sentiment analyzer
        sid = SentimentIntensityAnalyzer()

        # make dict of scores
        sentiments = {i : {'id': doc['id'], 
                           'compound_score': sid.polarity_scores(doc['text_en'])['compound']}
                      for i, doc in data.items()}

        # add category
        for k, v in sentiments.items():
            if sentiments[k]['compound_score'] <= -0.2:
                sentiments[k]['sentiment'] = 'negative'
            elif sentiments[k]['compound_score'] > 0.2:
                sentiments[k]['sentiment'] = 'positive'
            else:
                sentiments[k]['sentiment'] = 'neutral'
            
        return sentiments

if __name__ == '__main__':
    if len(sys.argv) > 2:
        try:
            kwargs = {kwarg.split("=")[0]: eval(kwarg.split("=")[1]) for kwarg in sys.argv[2:]}
        except NameError:
            kwargs = {kwarg.split("=")[0]: kwarg.split("=")[1].split(",") for kwarg in sys.argv[2:]}
        getattr(Analyse(), sys.argv[1])(**kwargs)
    elif len(sys.argv) == 2:
        getattr(Analyse(), sys.argv[1])()


NameError: name '__file__' is not defined

In [45]:
get_sentiments(all_tweets)

NameError: name 'get_sentiments' is not defined

In [ ]:
    with c1:
        st.title("Sentiment analysis")

        # Sentiment pie-charts
        #All data
        query_df = pd.DataFrame(query_data_general['hits'])
        politicians_df = pd.DataFrame(politicians['hits'])
        news_df = pd.DataFrame(news['hits'])
        all_tweets_df = pd.concat([query_df, politicians_df, news_df])

        colors_pie = ['#11E7FB', '#6082FD', '#731F7D']

        labels = ["Positive", "Neutral", "Negative"]

        values_query = [
            all_tweets_df['sentiment'].value_counts()['positive'],
            all_tweets_df['sentiment'].value_counts()['neutral'],
            all_tweets_df['sentiment'].value_counts()['negative']
        ]
        values_all = [41, 35, 24]

        # Create subplots: use 'domain' type for Pie subplot
        fig = make_subplots(rows=1,
                            cols=2,
                            specs=[[{
                                'type': 'domain'
                            }, {
                                'type': 'domain'
                            }]])
        fig.add_trace(
            go.Pie(title='Twitter',
                   labels=labels,
                   values=values_all,
                   direction='clockwise',
                   sort=False), 1, 1)
        fig.add_trace(
            go.Pie(title=query,
                labels=labels,
                   values=values_query,
                   direction='clockwise',
                   sort=False), 1, 2)

        # Use `hole` to create a donut-like pie chart
        fig.update_traces(hole=.4,
                          hoverinfo="label+percent",
                          textfont_size=15,
                          marker=dict(colors=colors_pie))

        fig.update_layout(
            # Add annotations in the center of the donut pies.
            annotations=[
                dict(text='',
                     x=0.18,
                     y=0.5,
                     font_size=18,
                     font_color='#731F7D',
                     showarrow=False),
                dict(text="",
                     x=0.82,
                     y=0.5,
                     font_size=18,
                     font_color='#731F7D',
                     showarrow=False)], autosize=False, width=500, height=250, margin=dict(l=0,r=0,b=0,t=0,pad=1))
        st.plotly_chart(fig)

In [30]:
def get_regulation():
    lexai_url = "http://127.0.0.1:7700/indexes/eurlex/search"
    result = requests.get(lexai_url, params=params, headers=headers).json()
    reg = []
    for i in result["hits"]:
        title = i["title"]
        author = i['author']
        date = pd.to_datetime(i['date']).date()
        link = i['link']
        reg.append({
            "title": title,
            "author": author,
            "date": date,
            "link": link
        })

    return reg


In [32]:
get_regulation()

[{'title': 'Opinion - European Economic and Social Committee - Health technology assessment / Compromise',
  'author': 'European Economic and Social Committee',
  'date': datetime.date(2021, 4, 27),
  'link': 'https://eur-lex.europa.eu/legal-content/AUTO/?uri=PI_EESC:EESC-2021-01784-AC'},
 {'title': 'Voting result Regulation on the European Institute of Innovation and Technology (EIT) Adoption of the legislative act 3792nd meeting of the COUNCIL OF THE EUROPEAN UNION (Foreign Affairs) 10 May 2021, Brussels',
  'author': 'Council of the European Union',
  'date': datetime.date(2021, 10, 5),
  'link': 'https://eur-lex.europa.eu/legal-content/AUTO/?uri=CONSIL:ST_8601_2021_INIT'},
 {'title': 'Statement of revenue and expenditure of the European Institute of Innovation and Technology for the financial year 2020 – amending budget No 3 2021/C 114/53',
  'author': 'European Institute of Innovation and Technology',
  'date': datetime.date(2021, 3, 31),
  'link': 'https://eur-lex.europa.eu/legal

In [34]:
lexai_url = "http://127.0.0.1:7700/indexes/eurlex/search"
result = requests.get(lexai_url, params=params, headers=headers).json()

In [39]:
regulation = get_regulation()

In [47]:
regulation[0:2]

[{'title': 'Opinion - European Economic and Social Committee - Health technology assessment / Compromise',
  'author': 'European Economic and Social Committee',
  'date': datetime.date(2021, 4, 27),
  'link': 'https://eur-lex.europa.eu/legal-content/AUTO/?uri=PI_EESC:EESC-2021-01784-AC'},
 {'title': 'Voting result Regulation on the European Institute of Innovation and Technology (EIT) Adoption of the legislative act 3792nd meeting of the COUNCIL OF THE EUROPEAN UNION (Foreign Affairs) 10 May 2021, Brussels',
  'author': 'Council of the European Union',
  'date': datetime.date(2021, 10, 5),
  'link': 'https://eur-lex.europa.eu/legal-content/AUTO/?uri=CONSIL:ST_8601_2021_INIT'}]

In [62]:
today = date.today()
initial_value_for_start_date = today + relativedelta(months=-12)
initial_value_for_end_date = today
start_date = pd.to_datetime(initial_value_for_start_date).date()
end_date = pd.to_datetime(initial_value_for_end_date).date()
regulation = get_regulation()

In [68]:
regulation_2= pd.DataFrame(regulation).sort_values(by="date", ascending=False )

In [95]:
regulation_2= regulation_2[(regulation_2["date"] >= start_date) & (regulation_2["date"] <= end_date)]

In [108]:
regulation_2=regulation_2.to_dict()

In [104]:
    with c3:
        '''
        ## Regulations
        '''
        st.title('Regulations')
        ## Range selector
        today = date.today()
        initial_value_for_start_date = today + relativedelta(months=-12)
        initial_value_for_end_date = today
        start_date, end_date = st.date_input(
            "Filter regulations by date: ",
            [initial_value_for_start_date, initial_value_for_end_date])
        start_date = pd.to_datetime(start_date).date()
        end_date = pd.to_datetime(end_date).date()
        regulation = get_regulation()
        expander = st.beta_expander("expand")
        with expander:
            for i in regulation:
                if i["date"] >= start_date and i["date"] <= end_date:
                    st.markdown(f'<p><strong><span style="color: rgb(96, 130, 253);">Title: </span></strong>{i["title"]}</p>',unsafe_allow_html=True)
                    st.markdown(f'<p><strong><span style="color: rgb(96, 130, 253;">Author: </span></strong>{i["author"]}</p>',unsafe_allow_html=True)
                    st.markdown(f'<p><strong><span style="color: rgb(96, 130, 253);">Date: </span></strong>{i["date"]}</p>',unsafe_allow_html=True)
                    if st.button('Link', key=i):
                        js = f"window.open('{i['link']}')"
                        html = '<img src onerror="{}">'.format(js)
                        div = Div(text=html)
                        st.bokeh_chart(div)
                    st.write('-----')

NameError: name 'c3' is not defined

In [120]:
regulation.sort(key=lambda item:item['date'], reverse=True)
regulation

[{'title': 'Voting result Regulation on the European Institute of Innovation and Technology (EIT) Adoption of the legislative act 3792nd meeting of the COUNCIL OF THE EUROPEAN UNION (Foreign Affairs) 10 May 2021, Brussels',
  'author': 'Council of the European Union',
  'date': datetime.date(2021, 10, 5),
  'link': 'https://eur-lex.europa.eu/legal-content/AUTO/?uri=CONSIL:ST_8601_2021_INIT'},
 {'title': "Draft Regulation of the European Parliament and of the Council establishing the European Cybersecurity Industrial, Technology and Research Competence Centre and the Network of National Coordination Centres (first reading) - Decision to use the written procedure for the adoption of the Council's position at first reading and of the statement of the Council's reasons",
  'author': 'Council of the European Union',
  'date': datetime.date(2021, 9, 4),
  'link': 'https://eur-lex.europa.eu/legal-content/AUTO/?uri=CONSIL:ST_7587_2021_INIT'},
 {'title': 'REGULATION OF THE EUROPEAN PARLIAMENT A

In [94]:
regulation_2[(regulation_2["date"] >= start_date) & (regulation_2["date"] <= end_date)]

,title,author,date,link
12,REGULATION OF THE EUROPEAN PARLIAMENT AND OF T...,"European Parliament, Council of the European U...",2021-05-20,https://eur-lex.europa.eu/legal-content/AUTO/?...
11,DECISION OF THE EUROPEAN PARLIAMENT AND OF THE...,"European Parliament, Council of the European U...",2021-05-20,https://eur-lex.europa.eu/legal-content/AUTO/?...
6,REGULATION OF THE EUROPEAN PARLIAMENT AND OF T...,"European Parliament, Council of the European U...",2021-05-20,https://eur-lex.europa.eu/legal-content/AUTO/?...
7,REGULATION OF THE EUROPEAN PARLIAMENT AND OF T...,Council of the European Union,2021-04-30,https://eur-lex.europa.eu/legal-content/AUTO/?...
4,Proposal for a Regulation of the European Parl...,Council of the European Union,2021-04-29,https://eur-lex.europa.eu/legal-content/AUTO/?...
10,Proposal for a REGULATION OF THE EUROPEAN PARL...,Council of the European Union,2021-04-28,https://eur-lex.europa.eu/legal-content/AUTO/?...
14,Opinion of the European Data Protection Superv...,Council of the European Union,2021-04-27,https://eur-lex.europa.eu/legal-content/AUTO/?...
0,Opinion - European Economic and Social Committ...,European Economic and Social Committee,2021-04-27,https://eur-lex.europa.eu/legal-content/AUTO/?...
17,Draft REGULATION OF THE EUROPEAN PARLIAMENT AN...,Council of the European Union,2021-04-20,https://eur-lex.europa.eu/legal-content/AUTO/?...
18,Draft REGULATION OF THE EUROPEAN PARLIAMENT AN...,Council of the European Union,2021-04-14,https://eur-lex.europa.eu/legal-content/AUTO/?...
